In [9]:
%matplotlib notebook
import pandas as pd
import numpy as np
import scipy.stats as st
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from collections import OrderedDict
import matplotlib.ticker as ticker

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df.transpose().head()
df.T.describe()

,1992,1993,1994,1995
count,3650.000000,3650.000000,3650.000000,3650.000000
mean,33312.107476,41861.859541,39493.304941,47743.550969
std,200630.901553,98398.356203,140369.925240,69781.185469
min,-717071.175466,-321586.023683,-450827.613097,-189865.963265
25%,-102740.398364,-26628.302213,-57436.397393,1774.555612
50%,29674.931050,43001.976658,41396.781369,49404.322978
75%,167441.838695,108296.577923,137261.713785,94164.333867
max,817505.608159,395586.505068,490091.665037,320826.888044


In [10]:
#Etremo del limite de confianza para la media con varianza desconocidad, 
#esto es un uso de la distribucion t-students. Se usa el paquete de estadistica de la libreria scipy
df['i_min'], df['i_max'] = st.t.interval(0.95, len(df.count(axis=1))-1, loc= df.mean(axis=1), scale = df.sem(axis=1)*1/8 )
df['mean'] = df.mean(axis=1)
df['std']  = df.std(axis=1)
df['sem']  = df.sem(axis=1)*1/8
df['yerr'] = df['i_max'].abs()-df['i_min'].abs()
print(df[['mean', 'i_min', 'i_max', 'yerr', 'std', 'sem']])

              mean         i_min         i_max         yerr            std  \
1992  33312.107476  31991.047635  34633.167316  2642.119681  200548.481103   
1993  41861.859541  41213.952784  42509.766298  1295.813514   98357.933533   
1994  39493.304941  38569.035202  40417.574680  1848.539478  140312.260383   
1995  47743.550969  47284.074782  48203.027157   918.952375   69752.518915   

             sem  
1992  414.693281  
1993  203.373937  
1994  290.129974  
1995  144.222196  


In [11]:
alpha_co = 0.7
cl_max_color='red'
cl_min_color='blue'
cl_neitral_color='grey'

class CursorE(object):
    _df = None
    _bl = None
    _num_bins = 8
    def __init__(self, ax, data_F, bars):
        self._df = data_F
        self._bl = bars
        self.ax  = ax
        self.lx  = ax.axhline(color='black')
        
    def _get_color(self, d_series, val):
        s = d_series(['i_min', 'i_max'])
        s['val'] = val
        s = pd.cut(s, bins = self._num_bins, labels = list(range(self._num_bins)), 
                   include_lowest=False, right=True)
        return self._colors[s['var']+1]
        
    def mouse_move(self, event):
        if not event.inaxes:
            return
        x,y = event.xdata, event.ydata
        self.lx.set_ydata(y)
        for index, row in self._df.iterrows():
            if row['i_max'] < y:
                self._bl[self._df.index.get_loc(index)].set_color(cl_min_color)
                continue
            if row['i_min'] > y:
                self._bl[self._df.index.get_loc(index)].set_color(cl_max_color)
                continue
            self._bl[self._df.index.get_loc(index)].set_color(cl_neitral_color)  
        plt.draw()
                  

In [12]:
def plot_base(fix_x, fig_y, fig_title, c_alpha=0.5):
    ax = df['mean'].plot.bar(yerr=df['yerr'], 
        title =fig_title, figsize=(fix_x, fig_y), 
        legend=False, fontsize=10, alpha=c_alpha, width=0.95, 
        rot=0, position=0, style='-', color=cl_neitral_color)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')

    majors = [0.5,1.5,2.5, 3.5]
    ax.xaxis.set_major_locator(ticker.FixedLocator(majors))

    plt.tight_layout()
    
    return (ax, ax.get_children()[1:5])


In [14]:
ax1, barlist1=plot_base(6, 6, 'Solution Exercise 3', c_alpha=0.5) 

bars_leg=[]

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.04), 
           handles=bars_leg, ncol=len(bars_leg), columnspacing=0.2, handletextpad=0.1, fontsize=7)

plt.tight_layout()
plt.show()

hcursor = CursorE(ax1, df, barlist1)
plt.connect('motion_notify_event', hcursor.mouse_move)

8